В этом ноутбуке:
- установка ClickHouse на Windows
- использование Clickhouse через консоль, Play UI, Python

Для установки ClickHouse на Windows 10 требуется версия Ubuntu с соответствующей библиотекой 

(см. https://packages.ubuntu.com/search?lang=en&keywords=clickhouse&searchon=names)

В данном случае устанавливалась Ubuntu 22.04.4 LTS

Источники:
- https://clickhouse.com/
- https://www.comss.ru/page.php?id=4897&ysclid=l5cowhpo9363429776
- https://clickhouse.com/docs/en/quick-start
- https://clickhouse.com/learn/lessons/gettingstarted/
- https://clickhouse-driver.readthedocs.io/_/downloads/en/latest/pdf/
- https://ivan-shamaev.ru/how-to-write-data-to-clickhouse-using-python/?ysclid=l5gzx16x8313881966

Прежде чем устанавливать Ubuntu нужно активировать компонент Windows Subsystem for Linux в Windows 10. Для этого, нажав клавиши Win + R нужно запустить команду appwiz.cpl, затем в открывшемся окне перейти на вкладку "Включение и отключение компонентов Windows" и поставить галочку на против нужного компонента: "Подсистема Windows для Linux"

После в Microsoft Store в поиске ввести Linux и скачать нужную версию Ubuntu

При первом запуске среды Linux вам будет предложено ввести имя пользователя и пароль UNIX. Они не должны совпадать с вашим именем пользователя и паролем Windows, и будут использоваться в среде Linux.

Вы можете запустить установленную среду Linux с помощью команды wsl. Если у вас установлено несколько дистрибутивов Linux, вы можете выбрать среду Linux по умолчанию, которую будет запускаться этой командой.


В WSL можно использовать привычные команды терминала Linux. Если вы привыкли к стандартной командной строке Windows с ее командами DOS, вот несколько основных команд, общих для Bash и Windows:

- Изменить директорию: cd в Bash, cd или chdir в DOS
- Показать содержимое директории: ls в Bash, dir в DOS
- Переместить или переименовать файл: mv в Bash, move и rename в DOS
- Копировать файл: cp а Bash, copy в DOS
- Удалить файл: rm в Bash, del или erase в DOS
- Создать директорию: mkdir in Bash, mkdir в DOS
- Использовать текстовые редактор: vi или nano в Bash, edit в DOS

Важно помнить, что, в отличие от Windows, оболочка Bash и имитирующая ее Linux среда чувствительны к регистру. Другими словами, файл с названием “File.txt” отличается от файла с названием “file.txt”.

# Развертывание ClickHouse

После установки Ubuntu на наш ноутбук с Windows 10 можно приступить к установке ClickHause. Cледуя шагам, предложенным в инструкции от Яндекса (https://clickhouse.com/), выполним ряд соответствующих команд в терминале Ubuntu:

In [ ]:
sudo apt-get install apt-transport-https ca-certificates dirmngr

In [ ]:
sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv 8919F6BD2B48D754

In [ ]:
echo "deb https://packages.clickhouse.com/deb stable main" | sudo tee \
    /etc/apt/sources.list.d/clickhouse.list
sudo apt-get update

In [ ]:
sudo apt-get install -y clickhouse-server clickhouse-client

# Работа с ClickHouse

Для запуска нужно выполнить следущие команды:

In [ ]:
sudo clickhouse start

# Через терминал

In [ ]:
clickhouse-client # or "clickhouse-client --password" if you set up a password.

In [ ]:
SELECT *
FROM helloworld.my_first_table
ORDER BY timestamp

In [ ]:
:) exit
Bye.

## Для работы через ClickHouse Play UI далее перейти по ссылке http://localhost:8123/play
- Если для пользователя вводился пароль, то ввести его в правом верхнем углу в поле password

![image_Play_UI](ClickHouse_Play_UI.JPG)

- SHOW DATABASES - Отображение существующих баз данных

Создание базы данных

In [ ]:
CREATE DATABASE IF NOT EXISTS hello

In [ ]:
CREATE TABLE hello.my_first_table
(
user_id       UInt32,
product       String,
datetime      DateTime,
customer      Nullable(String),
amount        Nullable(Float32)
)
ENGINE = MergeTree()
PRIMARY KEY (user_id, datetime)

Движки таблиц (ENGINE) и их особенности:

MergeTree - Наиболее универсальные и функциональные движки таблиц для задач с высокой загрузкой. Общим свойством этих движков является быстрая вставка данных с последующей фоновой обработкой данных. (MergeTree, ReplacingMergeTree, SummingMergeTree, AggregatingMergeTree, CollapsingMergeTree, VersionedCollapsingMergeTree, GraphiteMergeTree)

Log - Простые движки с минимальной функциональностью. Они наиболее эффективны, когда вам нужно быстро записать много небольших таблиц (до примерно 1 миллиона строк) и прочитать их позже целиком.(TinyLog, StripeLog, Log)

Движки для интеграции - Движки для связи с другими системами хранения и обработки данных. (Kafka, MySQL, ODBC, JDBC, S3)

Специальные движки: ODBC, JDBC, MySQL, MongoDB, HDFS, Kafka, EmbeddedRocksDB, RabbitMQ, PostgreSQL, Distributed, MaterializedView, Dictionary, Merge, File, Null, Set, Join, URL, View, Memory, Buffer.




In [ ]:
DESCRIBE hello.my_first_table

Добавление данных

In [ ]:
INSERT INTO helloworld.my_first_table (user_id, message, timestamp, metric) VALUES [(101, 'Hello, ClickHouse!',now(),-1.0),
    (102, 'Insert a lot of rows per batch',                     yesterday(), 1.41421 ),
    (102, 'Sort your data based on your commonly-used queries', today(),     2.718   ),
    (101, 'Granules are the smallest chunks of data read',      now() + 5,   3.14159 )]

In [ ]:
SELECT * FROM helloworld.my_first_table

## Работа через Python

In [ ]:
pip install clickhouse-driver

In [1]:
from clickhouse_driver import Client
from datetime import datetime

In [2]:
client = Client(host='localhost')

In [3]:
client.execute('SHOW DATABASES')

[('INFORMATION_SCHEMA',), ('default',), ('information_schema',), ('system',)]

In [6]:
client.execute('CREATE DATABASE IF NOT EXISTS hello')

[]

In [7]:
client.execute('CREATE TABLE hello.my_first_table (user_id INT, message String, datetime DateTime, metric Float32) ENGINE = Memory')

[]

In [8]:
client.execute("INSERT INTO hello.my_first_table (*) VALUES ( 1, 'product 1', '2021-10-01', 10)")

[]

In [9]:
client.execute('SELECT * FROM hello.my_first_table')

[(1, 'product 1', datetime.datetime(2021, 10, 1, 0, 0), 10.0)]

In [10]:
client.execute('DROP TABLE IF EXISTS hello.my_first_table')

[]

In [19]:
client.execute('DROP DATABASE hello')

[]

## Добавление таблицы из DataFrame

In [6]:
pip install clickhouse-driver[numpy]

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np

In [8]:
client = Client(host='localhost', settings={'use_numpy': True})

In [26]:
client.execute('CREATE DATABASE IF NOT EXISTS hello')

[]

In [27]:
client.execute('CREATE TABLE hello.my_first_table (user_id INT, product String, datetime DateTime, amount Float32) ENGINE = Memory')

[]

In [28]:
# Creating Dataframe 
df = pd.DataFrame([
                    [152, 'product 152', '2021-10-04', 4], 
                    [153, 'product 153', '2021-10-06', 5], 
                    [154, 'product 154', '2021-10-07', 7]
                  ],
                  columns =['user_id', 'product', 'datetime', 'amount'])
print(df)

   user_id      product    datetime  amount
0      152  product 152  2021-10-04       4
1      153  product 153  2021-10-06       5
2      154  product 154  2021-10-07       7


In [29]:
client.insert_dataframe(f'INSERT INTO hello.my_first_table VALUES', df)

3

In [30]:
client.query_dataframe('SELECT * FROM hello.my_first_table VALUES')

,user_id,product,datetime,amount
0,152,product 152,2021-10-04,4.0
1,153,product 153,2021-10-06,5.0
2,154,product 154,2021-10-07,7.0


In [31]:
client.execute('DROP DATABASE hello')

[]

## Вставка данных из csv

In [32]:
from csv import DictReader

In [33]:
def iter_csv(filename):
    converters = {
        'qty': int,
        'time': lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    }

    with open(filename, 'r') as f:
        reader = DictReader(f)
        for line in reader:
            yield {k: (converters[k](v) if k in converters else v) for k,v in line.items()}

In [34]:
client = Client('localhost')

In [35]:
client.execute('CREATE TABLE IF NOT EXISTS data_csv '
    '('
    'time DateTime, '
    'order String, '
    'qty Int32'
    ') Engine = Memory'
    )

[]

In [39]:
df = pd.DataFrame([
                    ['2021-10-04 15:23:14', 'product 152', 4], 
                    ['2021-10-06 06:14:45', 'product 153', 5], 
                    ['2021-10-07 12:20:32', 'product 154', 7]
                  ],
                  columns =['time', 'order', 'qty'])
df.to_csv('data.csv')

In [40]:
client.execute('INSERT INTO data_csv VALUES', iter_csv('data.csv'))

3

In [42]:
client.query_dataframe('SELECT * FROM data_csv VALUES')

,time,order,qty
0,2021-10-04 15:23:14,product 152,4
1,2021-10-06 06:14:45,product 153,5
2,2021-10-07 12:20:32,product 154,7


In [45]:
client.execute('DROP TABLE data_csv')

[]